# Data Exploration
* Data from: https://open.fda.gov/apis/drug/event/download/
* API: https://api.fda.gov/drug/event.json

In [1]:
import json

with open('data/testdata.json') as f:
    datajson = json.load(f)

In [5]:
import helper

df = helper.get_reports_df(datajson['results'])

In [6]:
df

,brand_name,generic_name,manufacturer_name,occurcountry,patientonsetage,patientonsetageunit,patientsex,product_type,reactionmeddrapt,reactionoutcome,receivedate,route,safetyreportid,serious,substance_name
0,SAXENDA,LIRAGLUTIDE,Novo Nordisk,US,NaN,NaN,2,HUMAN PRESCRIPTION DRUG,Gastrointestinal disorder,6,20190227,SUBCUTANEOUS,16013244,2,LIRAGLUTIDE
1,GENOTROPIN,SOMATROPIN,Pharmacia and Upjohn Company LLC,US,3,801,1,HUMAN PRESCRIPTION DRUG,Growth retardation,6,20190227,NaN,16013864,1,NaN
2,GENOTROPIN,SOMATROPIN,Pharmacia and Upjohn Company LLC,US,3,801,1,HUMAN PRESCRIPTION DRUG,Blood thyroid stimulating hormone decreased,6,20190227,NaN,16013864,1,NaN
3,GENOTROPIN,SOMATROPIN,Pharmacia and Upjohn Company LLC,US,3,801,1,HUMAN PRESCRIPTION DRUG,Product use in unapproved indication,6,20190227,NaN,16013864,1,NaN
4,GENOTROPIN,SOMATROPIN,Pharmacia and Upjohn Company LLC,US,3,801,1,HUMAN PRESCRIPTION DRUG,Fatigue,3,20190227,NaN,16013864,1,NaN
5,GENOTROPIN,SOMATROPIN,Pharmacia and Upjohn Company LLC,US,3,801,1,HUMAN PRESCRIPTION DRUG,Constipation,3,20190227,NaN,16013864,1,NaN
6,ATACAND,CANDESARTAN CILEXETIL,AstraZeneca Pharmaceuticals LP,DO,NaN,NaN,1,HUMAN PRESCRIPTION DRUG,Death,5,20190227,ORAL,16014487,1,CANDESARTAN CILEXETIL
7,COSOPT PF,DORZOLAMIDE HYDROCHLORIDE AND TIMOLOL MALEATE,"Akorn, Inc.",US,NaN,NaN,NaN,HUMAN PRESCRIPTION DRUG,Ocular hyperaemia,NaN,20190213,OPHTHALMIC,15958076,2,TIMOLOL MALEATE
8,COSOPT PF,DORZOLAMIDE HYDROCHLORIDE AND TIMOLOL MALEATE,"Akorn, Inc.",US,NaN,NaN,NaN,HUMAN PRESCRIPTION DRUG,Eye irritation,NaN,20190213,OPHTHALMIC,15958076,2,TIMOLOL MALEATE
9,ADVAIR DISKUS,FLUTICASONE PROPIONATE AND SALMETEROL,GlaxoSmithKline LLC,US,NaN,NaN,2,HUMAN PRESCRIPTION DRUG,Product storage error,6,20190214,RESPIRATORY (INHALATION),15958951,2,SALMETEROL XINAFOATE
